In [11]:
import numpy as np
import pickle
import os
import tqdm
import pandas as pd
import math

In [3]:
data_path = 'mathorcup_recom_listwise/data/'
user_path = 'data_splitted_by_user_id'

In [4]:
with open(data_path + 'train_clickeds.pickle', 'rb') as handle:
    train_clickeds = pickle.load(handle)

In [7]:
len(np.where(np.isnan(train_clickeds))[0])

1377783392

In [8]:
len(np.where(train_clickeds == 0)[0])

34801007

In [12]:
def ibcf(data, norm):
    user_num, item_num = data.shape
    result = data.copy()
    
    if norm:
        user_avg_rate = np.nanmean(data, axis=1)  # user averate rate
        result = data - user_avg_rate.reshape(-1, 1)
    
    print('start calculating item_sim')
    item_sim = np.ones([item_num, item_num])
    for i in tqdm.tqdm(range(item_num)):
        for j in range(i + 1, item_num):
            idxes = np.intersect1d(np.where(~np.isnan(data[:, i])), np.where(~np.isnan(data[:, j])))
            item_sim[i, j] = item_sim[j, i] = np.nansum(data[:, i] * data[:, j]) / \
            (math.sqrt(sum(data[_, i] ** 2 for _ in idxes)) * math.sqrt(sum(data[_, j] ** 2 for _ in idxes)))
    
    print('start calculating rate')
    rate = np.nanmean(result) * np.ones(data.shape)  # difference
    for i in tqdm.tqdm(range(item_num)):
        idxes = set(range(item_num)) - {i} - set(np.where(np.isnan(item_sim[i, :]))[0])
        if idxes:
            rate[:, i] = sum([item_sim[i, j] * data[:, j] for j in idxes]) / \
            sum([item_sim[i, j] for j in idxes])
    
    print('start calculating result')
    for i in tqdm.tqdm(range(item_num)):
        idxes = np.where(np.isnan(data[:, i]))
        result[idxes, i] =rate[idxes, i]
    
    if norm:
        result += user_avg_rate.reshape(-1, 1)
    return result

In [13]:
completed_train_clickeds = ibcf(train_clickeds, norm=True)

  0%|          | 0/1485 [00:00<?, ?it/s]

start calculating item_sim


<ipython-input-12-03bb7fbad797>:14: RuntimeWarning: invalid value encountered in true_divide
  item_sim[i, j] = item_sim[j, i] = np.nansum(data[:, i] * data[:, j]) / \
  0%|          | 2/1485 [04:13<52:14:44, 126.83s/it]


KeyboardInterrupt: 

In [ ]:
with open(data_path + 'completed_train_clickeds.pickle', 'wb') as handle:
    pickle.dump(completed_train_clickeds, handle, protocol=pickle.HIGHEST_PROTOCOL)